# Rigid Body Dynamics and Simulation

Finally, let's see how to start assembling the tools mentioned in the Systems and Symbolic/Autodiff tutorials to make robots do interesting things.

For these examples, we'll explore simulating the ultra-classic cart pole, pictured below.

<img src="https://danielpiedrahita.files.wordpress.com/2017/02/cart-pole.png" alt="drawing" style="width: 400px;"/>

As a more complete demo, we can create an LQR solution around that upright fixed point and simulate it!

See the quickstart guide for a written explanation of the many pieces of this.

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np

from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         LinearQuadraticRegulator, RigidBodyPlant,
                         RigidBodyTree, Simulator, SignalLogger, LeafSystem, PortDataType)
from underactuated import (PlanarRigidBodyVisualizer)
from IPython.display import HTML


In [2]:
import pdb

class SwingupController(LeafSystem):
    def __init__(self, rbt,
                 control_period=0.005,
                 print_period=0.5):
        LeafSystem.__init__(self)
        self.set_name("Swing up Controller")


       
        #self.B_inv = np.linalg.inv(self.B)
        # Copy lots of stuff
        self.rbt = rbt
        self.nq = rbt.get_num_positions()
        #self.plant = plant
        self.nu = rbt.get_input_port(0).size()
        #self.print_period = print_period
        self.last_print_time = -print_period
        self.shut_up = False

        self.robot_state_input_port = \
            self._DeclareInputPort(PortDataType.kVectorValued,
                                   rbt.get_num_positions() +
                                   rbt.get_num_velocities())
            

        self._DeclareContinuousState(self.nu)
        #self._DeclarePeriodicContinuousUpdate(period_sec=control_period)
        self._DeclareVectorOutputPort(
            BasicVector(self.nu),
            self._DoCalcVectorOutput)
        
    def _DoCalcVectorOutput(self, context, y_data):
        control_output = context.get_continuous_state_vector().get_value()
        y = y_data.get_mutable_value()
        # Get the ith finger control output
        # y[:] = control_output[:]
        y[:] = 0

In [3]:
#%tb
#%pdb
# Load in the cartpole from its URDF

tree = RigidBodyTree("underactuated/src/cartpole/cartpole.urdf",
                     FloatingBaseType.kFixed)

print tree

# Define an upright state
def UprightState():
    state = (0,math.pi,0,0)
    return state

def UprightPos():
    state = (math.pi/2,0)
    return state


def BalancingLQR(robot):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = robot.CreateDefaultContext()
    context.FixInputPort(0, BasicVector([0]))

    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())

    Q = np.diag((10., 10.,1,1))
    R = [1]

    return LinearQuadraticRegulator(robot, context, Q, R)
    
builder = DiagramBuilder()

robot = builder.AddSystem(RigidBodyPlant(tree))

controller = builder.AddSystem(BalancingLQR(robot))

#other_controller = builder.AddSystem(SwingupController(robot))

builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), robot.get_input_port(0))

logger = builder.AddSystem(SignalLogger(robot.get_output_port(0).size()))
logger._DeclarePeriodicPublish(1. / 30., 0.0)
builder.Connect(robot.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
context = simulator.get_mutable_context()

state = context.get_mutable_continuous_state_vector()
state.SetFromVector(UprightState() + 0.1*np.random.randn(4,))
simulator.StepTo(10.)

prbv = PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5])
ani = prbv.animate(logger, resample=30, repeat=True)
plt.close(prbv.fig)
HTML(ani.to_html5_video())

Spawning PlanarRigidBodyVisualizer for tree with 1 actuators


/usr/lib/python2.7/dist-packages/matplotlib/figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
